In [1]:
cd ../src/

/Users/syed/Desktop/ideal-stock-portfolio/src


In [5]:
import sqlalchemy
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as reader
import os

from database_setup_class import Database
from portfolio_class import Portfolio
from assets_class import Asset

import statsmodels.api as sm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def yf_retrieve_data(ticker_list,date_range):
    data_frames = []
    for ticker in ticker_list:
        data = yf.download(ticker, start=date_range[0], end=date_range[1])
        data_frames.append(data)
    return data_frames

In [6]:
database_name = 'DJIA'
djia = Database(database_name)


if not os.path.exists(f'../data/{database_name}.db'):
    date_range = ['2010-01-01', '2022-01-01']
    ticker_list = get_ticker_symbols('DJIA') + ['^GSPC']
    # Create database instance
    djia = Database('DJIA')
    djia.add_stock_to_database(ticker_list, start= date_range[0], end=date_range[1])
    djia.update_database()
    
else:
    djia.update_database()
    ticker_list = djia.get_ticker_list()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
market_data = data[0].reset_index()
market_data = market_data['Close'].pct_change().apply(lambda x: np.log(1+x)).dropna()

In [ ]:
market_data.columns = ['^GSPC']
market_data

In [ ]:
dow_jones = Database('DJIA')
ticker_symbols = dow_jones.get_ticker_symbols()
stock_data = dow_jones.get_dataframe(date_range = date_range)

In [ ]:
portfolio = Portfolio([Asset({"name": ticker, "data": stock_data[i] }) for i,ticker in enumerate(ticker_symbols[:4]) ])

In [ ]:
asset_frame = portfolio.assets_daily_returns

# Regression Model

In [ ]:
beta_assets = []
for col in portfolio.assets_daily_returns.iloc[:,1:]:
    covariance = portfolio.assets_daily_returns[col].cov(market_data)
    variance_market = market_data.var()
    beta_assets.append(covariance/variance_market)

In [ ]:
beta_assets

In [ ]:
X = market_data

stats_results = []
for col in portfolio.assets_daily_returns.iloc[:,1:]:
    
    y = portfolio.assets_daily_returns.iloc[:,1:][col]
    X_sm = sm.add_constant(X)
    model = sm.OLS(y, X_sm)
    results = model.fit()
    stats_results.append(results)

betas = [result.params['Close'] for result in stats_results]
betas

# Risk free rate

In [ ]:
from datetime import datetime
# 10 YEAR RATE FOR Treasury Constant Maturity Rate

T_rate = pd.read_csv('../data/DGS10.csv')
def get_treasury_rate(T_rate, asset_date_list):
    count = 0
    rate = []
    for i in T_rate['DGS10']:
        if i =='.':
            da.append(T_rate['DATE'][count])
            T_rate['DGS10'][count] = T_rate['DGS10'][count-1]        
        else:
            T_rate['DGS10'][count] = float(T_rate['DGS10'][count])

        if T_rate['DATE'][count] in asset_date_list:
            rate.append(T_rate['DGS10'][count]/100)
        count+=1
        
    return rate
        
asset_date_list = portfolio.assets_daily_returns['Date'].tolist()
ten_yr_rate = get_treasury_rate(T_rate, asset_date_list)

In [ ]:
asset_frame["10_year_rate"] = ten_yr_rate


In [ ]:
rf = reader.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', date_range[0], date_range[1])[0]

In [ ]:
rf = rf.reset_index()
rf

In [ ]:
rr = [r.strftime('%Y-%m-%d') for r in rf['Date']]
rr

In [ ]:

from datetime import datetime
d = []
for date in asset_frame['Date'].values:
    if date not in rr :
        d.append(date)
        

In [ ]:
rff = reader.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', '2021-12-01', '2022-01-18')[0]

In [ ]:
rff